In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

## Constants

In [ ]:
IMG_SIZE = (64, 64)  # Image size to resize to
BATCH_SIZE = 32      # Batch size for training and testing
EPOCHS = 30          # Number of epochs
NUM_CLASSES = 131    # Number of output classes

# 1. Data Preprocessing

In [ ]:
# Data augmentation and normalization for the training set
train_datagen = ImageDataGenerator(
    rescale=1./255,           # Normalize pixel values to [0, 1]
    shear_range=0.2,          # Apply random shearing transformations
    zoom_range=0.2,           # Apply random zoom
    horizontal_flip=True      # Randomly flip images horizontally
)

In [ ]:
# Load training data
training_set = train_datagen.flow_from_directory(
    'Training',              # Directory containing training data
    target_size=IMG_SIZE,    # Resize images to the target size
    batch_size=BATCH_SIZE,   # Batch size
    class_mode='categorical' # Multiclass classification
)

In [ ]:
# Data normalization for the test set
test_datagen = ImageDataGenerator(rescale=1./255)

# Load test data
test_set = test_datagen.flow_from_directory(
    'Test',                  # Directory containing test data
    target_size=IMG_SIZE,    # Resize images to the target size
    batch_size=BATCH_SIZE,   # Batch size
    class_mode='categorical',# Multiclass classification
    shuffle=False            # Do not shuffle test data for evaluation
)

# 2. Model Building

In [ ]:
# Initialize a sequential model
cnn = tf.keras.models.Sequential()

In [ ]:
# Add a convolutional layer with max pooling
cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu', input_shape=(64, 64, 3)))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

In [ ]:
# Add a second convolutional layer with max pooling
cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

In [ ]:
# Add a dropout layer to prevent overfitting
cnn.add(tf.keras.layers.Dropout(0.5))

# Flatten the feature maps into a 1D vector
cnn.add(tf.keras.layers.Flatten())

# Add a dense (fully connected) layer with 256 neurons
cnn.add(tf.keras.layers.Dense(units=256, activation='relu'))

# Add an output layer with softmax activation for multiclass classification
cnn.add(tf.keras.layers.Dense(units=NUM_CLASSES, activation='softmax'))

# Compile the model
cnn.compile(
    optimizer='rmsprop',                  # Optimizer
    loss='categorical_crossentropy',      # Loss function for multiclass classification
    metrics=['accuracy']                  # Evaluation metric
)


# 3. Model Training

In [ ]:
training_history = cnn.fit(
    x=training_set,
    validation_data=test_set,
    epochs=EPOCHS
)

# 4. Save the Trained Model

In [ ]:
cnn.save("trained_model.h5")

# 5. Evaluate and Visualize Results

## Print available metrics

In [ ]:
print("Training History Keys:", training_history.history.keys())

## Print final test set accuracy

In [ ]:
final_val_accuracy = training_history.history['val_accuracy'][-1]
print(f'Test set Accuracy: {final_val_accuracy * 100:.2f} %')

## Print final test set accuracy

In [ ]:
final_val_accuracy = training_history.history['val_accuracy'][-1]
print(f'Test set Accuracy: {final_val_accuracy * 100:.2f} %')

## Plot training accuracy

In [ ]:
epochs = range(1, EPOCHS + 1)
plt.plot(epochs, training_history.history['accuracy'], color='red', label='Training Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training Accuracy Over Epochs')
plt.legend()
plt.show()

## Plot validation accuracy

In [ ]:
plt.plot(epochs, training_history.history['val_accuracy'], color='cyan', label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Validation Accuracy Over Epochs')
plt.legend()
plt.show()
